In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
raw_usage = pd.read_csv("C:\\Users\\andre\\Documents\\StanfordMS\\CS 341\\Data\\train_usage.csv")
raw_revenue = pd.read_csv("C:\\Users\\andre\\Documents\\StanfordMS\\CS 341\\Data\\train_revenue.csv") 

Then, discard all missing data for simplicity. 

In [6]:
missing = np.asarray([True]*raw_usage.shape[0])
for i in range(raw_usage.shape[0]):
    if -1 in list(raw_usage.ix[i]): missing[i] = False
train_usage = raw_usage.ix[missing]
train_mean = train_usage.ix[:,6:].mean(axis=1)
train_usage['log_train_mean'] = np.log(train_mean) #take the logarithm 

C:\Users\andre\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Because there are 4 different metrics in the original data frame, I created 4 data frames for them respectively:

In [7]:
u_list = [0]*4
for i in range(4):
    u_list[i]= train_usage[train_usage.metric==i+1]

I first look at the difference between metrics of ipad and iphone apps:

In [23]:
for i in range(4):
    plt.subplot(2,2,i+1)
    u_list[i].ix[u_list[i].device=='ipad',6:14].mean().plot()
    u_list[i].ix[u_list[i].device=='iphone',6:14].mean().plot()
    plt.title("Metric"+str(i+1))
    plt.legend(['ipad','iphone'],loc='center',numpoints=1)
plt.show()


The above plot shows that each metric for ipad apps is on average larger than that for iphone apps.
It can also be seen that metrics do not change significantly over time.

Then I look at the metrics across different categories:

In [8]:
cat_group = list(set(train_usage['category'])) #the list of categories
cat_group.sort()

Then for each metric, I make a histogram for each catogory: 

In [24]:
for j in range(4):
    for i in range(23):
           plt.subplot(4,6,i+1)
           u_list[j].ix[u_list[j].category == cat_group[i],'log_train_mean'].hist()
           plt.title(cat_group[i])
    plt.show()

It can be seen that patterns of distribution of metrics among categories are similar, that is, some categories always have larger metrics than some others, like Utilities and Finance.

Next, for each app, I fit a quadratic model to each metric respectively and get the coefficients of models. Coeffiencts can be seen as an indicator of the extent of fluctuation.  

In [25]:
for j in range(4):
    u_list[j]["coef_0"] = [0.0]*u_list[j].shape[0]
    u_list[j]["coef_1"] = [0.0]*u_list[j].shape[0]
    u_list[j]["coef_2"] = [0.0]*u_list[j].shape[0]
    for k in range(len(cat_group)): u_list[j][cat_group[k]] = [0]*u_list[j].shape[0]

    for i in range(u_list[j].shape[0]):
        Y = np.array(u_list[j].iloc[i,range(6,14)])
        X = np.array(range(8))
        '''cat = np.array([0]*23)
        cat[cat_group.index(u_list[0].ix[i,'category'])] = 1'''
        c = np.polyfit(X,Y,2)
        u_list[j]["coef_0"].values[i] = c[2] 
        u_list[j]["coef_1"].values[i] = c[1]
        u_list[j]["coef_2"].values[i] = c[0]    
        for k in range(len(cat_group)):
            if cat_group[k]==u_list[j]["category"].values[i]: 
                u_list[j][cat_group[k]].values[i]=1    

C:\Users\andre\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\andre\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\andre\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

Then I take the average of those coefficients grouped by categories and make several plots: 

In [26]:
for i in range(4):
    plt.subplot(4,3,3*i+1)
    plt.bar(range(23),u_list[i]["coef_0"].groupby(u_list[i]["category"]).mean())
    plt.xticks(range(23),cat_group)
    plt.title("Constant term for metric "+str(i+1))
    
    plt.subplot(4,3,3*i+2)
    plt.bar(range(23),u_list[i]["coef_1"].groupby(u_list[i]["category"]).mean())
    plt.xticks(range(23),cat_group)
    plt.title("Linear term for metric "+str(i+1))
    
    plt.subplot(4,3,3*i+3)
    plt.bar(range(23),u_list[i]["coef_2"].groupby(u_list[i]["category"]).mean())
    plt.xticks(range(23),cat_group)
    plt.title("Quadratic term for metric "+str(i+1))
plt.show()

It can be seen that pattern of coefficients across categories is similar. Some categories always have larger coefficients while some others do not. 
Constant term can be seen as the initial level of a metric
Linear term can be seen as the intial potential of a metric (the derivative)

Furthermore, we can compute the derivative of different degrees of the fitted curve at various timepoints for more information.
In this sense, we can place more weight on those "important" categories when making prediction of the download of apps.   

Let's look at revenue:

In [ ]:
missing = np.asarray([True]*raw_revenue.shape[0])
for i in range(raw_revenue.shape[0]):
    if -1 in list(raw_revenue.ix[i]): missing[i] = False
train_revenue = raw_revenue.ix[missing]

rev_group = list(set(train_revenue["category"]))
rev_group.sort()
for i in range(len(rev_group)):
    cur_rev_mean = train_revenue.ix[train_revenue.category==rev_group[i],5:].mean(axis=1)
    plt.subplot(4,6,i+1)
    cur_rev_mean.hist(bins=50)
    plt.title(rev_group[i])
plt.show()

It is obvious that distribution of revenue is different across different categories.

In [ ]:
train_revenue["coef_0"] = [0.0]*train_revenue.shape[0] 
train_revenue["coef_1"] = [0.0]*train_revenue.shape[0] 
train_revenue["coef_2"] = [0.0]*train_revenue.shape[0] 
for i in range(train_revenue.shape[0]):
        Y = np.array(train_revenue.iloc[i,range(5,61)])
        X = np.array(range(56))
        c = np.polyfit(X,Y,2)
        train_revenue["coef_0"].values[i] = c[2] 
        train_revenue["coef_1"].values[i] = c[1]
        train_revenue["coef_2"].values[i] = c[0]     

plt.subplot(1,3,1)
plt.bar(range(len(rev_group)),train_revenue["coef_0"].groupby(train_revenue["category"]).mean())
plt.xticks(range(len(rev_group)),rev_group)
plt.title("Constant term")
    
plt.subplot(1,3,2)
plt.bar(range(len(rev_group)),train_revenue["coef_1"].groupby(train_revenue["category"]).mean())
plt.xticks(range(len(rev_group)),rev_group)
plt.title("Linear term")
    
plt.subplot(1,3,3)
plt.bar(range(len(rev_group)),train_revenue["coef_2"].groupby(train_revenue["category"]).mean())
plt.xticks(range(len(rev_group)),rev_group)
plt.title("Quadratic term")
plt.show()

C:\Users\andre\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\andre\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\andre\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [20]:
u_list[0]

,Unnamed: 0,id,name,category,device,metric,3/1/2015,3/8/2015,3/15/2015,3/22/2015,...,News,Photo and Video,Productivity,Reference,Shopping,Social Networking,Sports,Travel,Utilities,Weather
0,0,281704574,"AIM: Chat, Free Text, Photo Share, Voice Message",Social Networking,iphone,1,0.031330,0.033759,0.029534,0.031678,...,0,0,0,0,0,1,0,0,0,0
9,9,281935788,Epocrates References & Tools for Healthcare Pr...,Medical,iphone,1,0.126387,0.128059,0.133475,0.130418,...,0,0,0,0,0,0,0,0,0,0
18,18,284147312,iTrans DC Metro,Navigation,iphone,1,0.018817,0.021744,0.022590,0.025675,...,0,0,0,0,0,0,0,0,0,0
27,27,284809790,Yellow Pages,Navigation,iphone,1,0.033958,0.040568,0.032410,0.034834,...,0,0,0,0,0,0,0,0,0,0
36,36,284901416,AP Mobile,News,ipad,1,0.204284,0.163419,0.180931,0.186775,...,1,0,0,0,0,0,0,0,0,0
45,45,284962281,Aftonbladet,News,iphone,1,0.189983,0.194005,0.197961,0.210978,...,1,0,0,0,0,0,0,0,0,0
63,63,284963557,tv.nu,Utilities,iphone,1,0.136324,0.141294,0.139652,0.150903,...,0,0,0,0,0,0,0,0,1,0
72,72,284963764,hitta.se,Navigation,iphone,1,0.068887,0.061496,0.065507,0.047917,...,0,0,0,0,0,0,0,0,0,0
81,81,284967245,Eniro,Navigation,iphone,1,0.038432,0.029284,0.038872,0.036474,...,0,0,0,0,0,0,0,0,0,0
90,90,284971745,BVG FahrInfo Plus Berlin,Travel,iphone,1,0.034644,0.038024,0.031856,0.031463,...,0,0,0,0,0,0,0,1,0,0


The plots above indicate some underlied pattern of distribution of the data.
We can measure the importance of a category by its contribution to a certain metric (like revenue)